<a href="https://colab.research.google.com/github/stefaniemoara/dojo-clean-code/blob/master/REOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# !pip install gurobipy
import json
from gurobipy import Model, GRB

# Parse the JSON data
f = open('/KIRO-tiny.json')
data = json.load(f)
params = data["parameters"]
nb_jobs = params["size"]["nb_jobs"]
nb_tasks = params["size"]["nb_tasks"]
nb_machines = params["size"]["nb_machines"]
nb_operators = params["size"]["nb_operators"]
unit_penalty = params["costs"]["unit_penalty"]
tardiness = params["costs"]["tardiness"]
jobs = data["jobs"]
tasks = data["tasks"]

# Create a Gurobi model
model = Model()

# Add decision variables
start = {} # starting time of each task
assign_machine = {} # assignment of tasks to machines
assign_operator = {} # assignment of tasks to operators
for i in range(nb_tasks):
    start[i] = model.addVar(vtype=GRB.INTEGER, name="start_task_{}".format(i))
    for m in range(nb_machines):
        assign_machine[i, m] = model.addVar(vtype=GRB.BINARY, name="assign_task_{}_to_machine_{}".format(i, m))
    for o in range(nb_operators):
        assign_operator[i, o] = model.addVar(vtype=GRB.BINARY, name="assign_task_{}_to_operator_{}".format(i, o))

# Define binary variable y
y = model.addVar(vtype=GRB.BINARY)

# Define binary variable y
x = model.addVar(vtype=GRB.BINARY)


# Add the objective function
obj = 0
# print (tasks)
#for j in range(nb_jobs):
    # Compute completion time of job j
#    completion_time = 0
#    sequence = jobs[j]["sequence"]
#    for i in sequence:
#      completion_time += tasks[i]["processing_time"]
for j in range(nb_jobs):
    # Compute completion time of job j
    completion_time = 0
    sequence = jobs[j]["sequence"]
    for i in sequence:
      if i < len(tasks):
        completion_time += tasks[i]["processing_time"]
    completion_time += start[sequence[0]]
    # Compute tardiness of job
    # Define tardiness_time as a linear function of y
    tardiness_time = y * (completion_time - jobs[j]["due_date"])
    # Compute unit penalty for job
    unit_penalty_time = y
    # Add weighted values to the objective function
    weight = jobs[j]["weight"]
    obj += weight * (completion_time + unit_penalty * unit_penalty_time + tardiness * tardiness_time)

# Set the objective of the model
model.setObjective(obj, sense=GRB.MINIMIZE)

# Add constraints
for i in range(nb_tasks):
    # Constraint on completion time of task i
  model.addConstr(start[i] + completion_time[i] + tasks[i]["processing_time"] == completion_time[i])

# Add constraints
for j in range(nb_jobs):
    # Constraint on starting time of job j
  sequence = jobs[j]["sequence"]
  model.addConstr(start[sequence[0]] == job_start[j], name="start_time_job_{}".format(j))

# Define constraint on y
model.addConstr(y == 0 or y == 1)

# Define constraint on x
model.addConstr(x == 0 or x == 1)

TypeError: ignored